# Imports

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# Helper Functions

In [ ]:
"""Some helper functions for project 1."""
def load_csv_data(data_path, sub_sample=False):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y == "b")] = -1

    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids


def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, "w") as csvfile:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({"Id": int(r1), "Prediction": int(r2)})

# Test Functions

In [ ]:
RTOL=1e-4
ATOL=1e-8

MAX_ITERS = 2
GAMMA = 0.1

def initial_w_testing():
    return np.array([[0.5], [1.0]])

def y_testing():
    return np.array([[0.1], [0.3], [0.5]])

def tx_testing():
    return np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])

def test_least_squares(y, tx):
    w, loss = least_squares(y, tx)

    expected_w = np.array([[0.218786], [-0.053837]])
    expected_loss = 0.026942

    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape


def test_ridge_regression_lambda0(y, tx):
    lambda_ = 0.0
    w, loss = ridge_regression(y, tx, lambda_)

    expected_loss = 0.026942
    expected_w = np.array([[0.218786], [-0.053837]])

    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape


def test_ridge_regression_lambda1(y, tx):
    lambda_ = 1.0
    w, loss = ridge_regression(y, tx, lambda_)

    expected_loss = 0.03175
    expected_w = np.array([[0.054303], [0.042713]])

    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape


def test_logistic_regression_0_step(y, tx):
    expected_w = np.array([[0.463156], [0.939874]])
    y = (y > 0.2) * 1.0
    w, loss = logistic_regression(y, tx, expected_w, 0, GAMMA)

    expected_loss = 1.533694

    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape


def test_logistic_regression(y, tx, initial_w):
    y = (y > 0.2) * 1.0
    w, loss = logistic_regression(
        y, tx, initial_w, MAX_ITERS, GAMMA
    )

    expected_loss = 1.348358
    expected_w = np.array([[0.378561], [0.801131]])

    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape
    
def test_reg_logistic_regression(y, tx, initial_w):
    lambda_ = 1.0
    y = (y > 0.2) * 1.0
    w, loss = reg_logistic_regression(
        y, tx, lambda_, initial_w, MAX_ITERS, GAMMA
    )

    expected_loss = 0.972165
    expected_w = np.array([[0.216062], [0.467747]])

    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape


def test_reg_logistic_regression_0_step(y, tx):
    lambda_ = 1.0
    expected_w = np.array([[0.409111], [0.843996]])
    y = (y > 0.2) * 1.0
    w, loss = reg_logistic_regression(
        y, tx, lambda_, expected_w, 0, GAMMA
    )

    expected_loss = 1.407327

    np.testing.assert_allclose(loss, expected_loss, rtol=RTOL, atol=ATOL)
    np.testing.assert_allclose(w, expected_w, rtol=RTOL, atol=ATOL)
    assert loss.ndim == 0
    assert w.shape == expected_w.shape

# Our Implementations

## Least Squares

In [ ]:
def compute_mse(y, tx, w):
    """compute the loss by mse.
    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
        w: weights, numpy array of shape(D,), D is the number of features.
    
    Returns:
        mse: scalar corresponding to the mse with factor (1 / 2 n) in front of the sum

    >>> compute_mse(np.array([0.1,0.2]), np.array([[2.3, 3.2], [1., 0.1]]), np.array([0.03947092, 0.00319628]))
    0.006417022764962313
    """
    
    e = y - tx.dot(w)
    
    ## np.linalg.norm(e) ** 2 replaces e.dot(e)
    mse = (np.linalg.norm(e) ** 2) / (2 * len(e))
    return mse

In [ ]:
def least_squares(y, tx):
    """Calculate the least squares solution.
       returns mse, and optimal weights.
    
    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
    
    Returns:
        w: optimal weights, numpy array of shape(D,), D is the number of features.
        mse: scalar.
    """
    # w = (np.linalg.inv(tx.transpose() @ tx)) @ (tx.transpose() @ y)
    w = np.linalg.solve(tx.transpose() @ tx, tx.transpose() @ y)
    mse = compute_mse(y, tx, w)
    
    return w, mse

In [ ]:
test_least_squares(y_testing(), tx_testing())

## Ridge Regression

In [ ]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression.
    
    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
        lambda_: scalar.
    
    Returns:
        w: optimal weights, numpy array of shape(D,), D is the number of features.
    """
    D = 1 if len(tx.shape) == 1 else tx.shape[1]
    N = len(y)
    a = np.array((tx.transpose() @ tx) + (2 * N * lambda_) * np.eye(D))
    b = np.array(tx.transpose() @ y)
    
    w = np.linalg.inv(a) @ b
        
    return w, compute_mse(y, tx, w)

In [ ]:
test_ridge_regression_lambda0(y_testing(), tx_testing())
test_ridge_regression_lambda1(y_testing(), tx_testing())

## Logistic Regression

In [ ]:
## Passes the tests! ^^

def sigmoid(t):
    """apply sigmoid function on t.

    Args:
        t: scalar or numpy array

    Returns:
        scalar or numpy array
    """
    return 1 / (1 + np.exp(-t))

def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1) 

    Returns:
        a hessian matrix of shape=(D, D) 
    """
    
    N = len(y)
    sig = sigmoid(tx @ w)
    diag = sig * (1-sig) #np.zeros((N, N))
    #np.fill_diagonal(diag,  sig * (1 - sig))
                  
    return (1 / N) * (tx.T @ (diag * tx))

def calculate_loss(y, tx, w, penalty=0):
    """compute the cost by negative log likelihood.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1) 

    Returns:
        a non-negative loss
    """
    assert y.shape[0]  == tx.shape[0]
    assert tx.shape[1] == w.shape[0]
    
    sig = sigmoid(tx @ w) 
    left  = y * np.log(sig)
    right = (1-y) * np.log(1 - sig)
    
    return - np.mean(left + right) + penalty

def calculate_gradient(y, tx, w, lambda_=0):
    """compute the gradient of loss.
    
    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1) 

    Returns:
        a vector of shape (D, 1)
    """
    
    sig = sigmoid(tx @ w)
    
    ## last term is for adding a lambda_ * ||w||^2 penalty 
    return (1 / len(y)) * (tx.T @ (sig - y)) + (2 * lambda_ * w)

## Uses gradient descent
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    # init parameters
    threshold = 1e-8
    losses = []

    w = initial_w

    # start the logistic regression with GD
    for iter in range(max_iters):
        # get loss and update w.
        loss = calculate_loss(y, tx, w) 
        w = w - gamma * calculate_gradient(y, tx, w)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    
    
    losses.append(calculate_loss(y, tx, w))
    print("loss={l}".format(l=losses[-1]))
    
    return w, losses[-1]

In [ ]:
test_logistic_regression_0_step(y_testing(), tx_testing())
test_logistic_regression(y_testing(), tx_testing(), initial_w_testing())

## Regularized Logistic Regression

In [ ]:
def compute_penalty_term(lambda_, w):
    return lambda_ * (np.linalg.norm(w) ** 2)

def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    # init parameters
    threshold = 1e-8
    losses = []

    w = initial_w

    # start the logistic regression with GD
    for iter in range(max_iters):
        # get loss and update w.
        loss = calculate_loss(y, tx, w, penalty=compute_penalty_term(lambda_, w)) 
        w = w - gamma * calculate_gradient(y, tx, w, lambda_=lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    
    
    losses.append(calculate_loss(y, tx, w))
    print("loss={l}".format(l=losses[-1]))
    
    return w, losses[-1]

In [ ]:
test_reg_logistic_regression_0_step(y_testing(), tx_testing())
test_reg_logistic_regression(y_testing(), tx_testing(), initial_w_testing())

# Loading Higgs Model data

In [ ]:
def split_data(x, y, ratio, seed=1):
    """Split the dataset between train and test based on the split ratio."""
    np.random.seed(seed)
    
    n = len(y)
    indices = np.random.permutation(n)
    split = int(ratio * n)
    train_indices, test_indices = indices[:split], indices[split:]
    return x[train_indices], y[train_indices], x[test_indices], y[test_indices]

In [ ]:
y_test,  tx_test,  ids_test  = load_csv_data("D:/Downloads/test.csv")
y_train, tx_train, ids_train = load_csv_data("D:/Downloads/train.csv")

x_train, y_train, x_test, y_test = split_data(tx_train, y_train, 0.8)

N, D = x_train.shape

print(f'Number of samples: {N}')
print(f'Number of features: {D}')

In [ ]:
y_train = np.reshape(y_train, (len(y_train), 1))
y_test  = np.reshape(y_test, (len(y_test), 1))

# Binary classification

## Feature Expansion

In [ ]:
def standardize_data(x):
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)

def build_poly(x, degree):
    return np.array([xi ** np.arange(2, degree+1) for xi in x])

def append_poly_for_feature(x, feature_idx, degree):
    x_feature = x[:, feature_idx]
    polys = build_poly(x_feature, degree)
    
    return np.column_stack((x, polys))

def expand_features(x, degree):
    N, D = x.shape
    for i in np.arange(D):
        x = append_poly_for_feature(x, i, degree)
        
    return x

def expand_features_random_func(x, nb_features, func):
    for i in np.arange(nb_features):
        for j in np.arange(i+1, nb_features):
            feature_i = x[:, i]
            feature_j = x[:, j]
            N = len(feature_i)
            x = np.column_stack((x, np.array([func(feature_i[idx], feature_j[idx]) for idx in np.arange(N)])))
    
    return x

def add_jet_binary_features(x, jet_idx):
    jet_vals = x[:, jet_idx]
    res1 = (jet_vals == 0).astype(int)
    res2 = (jet_vals <= 1).astype(int)
    res3 = (jet_vals >= 2).astype(int)
    
    for res in [res1, res2, res3]:
        x = np.column_stack((x, res))
        
    return x

def truncated_fourier_term(x, j):
    assert j > 0
    
    return np.sin(j*np.pi*x) if j%2==0 else np.cos(j*np.pi*x)

def append_fourier_for_feature(x, degree, feature_idx):
    x_feature = x[:, feature_idx]
    
    for j in np.arange(1, degree+1):
        x = np.column_stack((x, truncated_fourier_term(x_feature, j)))
    
    return x

def append_fourier(x, degree, nb_features):
    for i in np.arange(nb_features):
        x = append_fourier_for_feature(x, degree, i)
        
    return x

def my_sum(x, y):
    return x + y

def my_mult(x, y):
    return x * y

def my_combo(x, y):
    return np.sin(x+y) + np.cos(x+y)

In [ ]:
degree_expansion    = True
jet_binary_features = False
fourier_expansion   = False
logs                = True
my_mult_expansion   = True
my_sum_expansion    = False
my_combo_expansion  = True

if(degree_expansion):
    x_train = expand_features(x_train, 4)
    x_test  = expand_features(x_test, 4)

if(jet_binary_features):
    x_train = add_jet_binary_features(x_train, 22)
    x_test = add_jet_binary_features(x_test, 22)
    
if(fourier_expansion):
    x_train = append_fourier(x_train, 4, 30)
    x_test = append_fourier(x_test, 4, 30)
    
if(logs):
    x_train = np.concatenate((x_train, np.log(x_train[:, [2, 9, 10, 13, 16, 19, 21]])), axis=1)
    x_test = np.concatenate((x_test,   np.log(x_test[:, [2, 9, 10, 13, 16, 19, 21]])), axis=1)

if(my_mult_expansion):
    x_train = expand_features_random_func(x_train, 30, my_mult)
    x_test = expand_features_random_func(x_test, 30, my_mult)
    
if(my_sum_expansion):
    x_train = expand_features_random_func(x_train, 30, my_sum)
    x_test = expand_features_random_func(x_test, 30, my_sum)
    
if(my_combo_expansion):
    x_train = expand_features_random_func(x_train, 30, my_sum)
    x_test = expand_features_random_func(x_test, 30, my_combo)

#jets_train = x_train[:, 22]
#jets_test  = x_test[:, 22]

x_train = standardize_data(x_train)
x_test  = standardize_data(x_test)

#x_train[:, 22] = jets_train
#x_test[:, 22]  = jets_test

print(x_train.shape)
print(x_test.shape)

## Removing "weird" features (inactive)

In [ ]:
## removing weirdness score >= 60

# Normalize each feature
#tx_normalised = standardize_data(tx_train)
#weirdness = np.sum(np.abs(tx_normalised), axis=1)
#plt.boxplot(weirdness)
#plt.show()

#y_train = y_train[weirdness < 60]
#x_train = x_train[weirdness < 60]

## Our models: Starting with Least Squares, Ridge Regression

In [ ]:
def compute_accuracy(truth, predictions):
    return len(np.where(truth == predictions)[0]) / len(truth)

In [ ]:
def transform_prediction(prediction): 
    prediction[np.where(prediction>0)[0]]  = 1
    prediction[np.where(prediction<=0)[0]] = -1
    
    return prediction

In [ ]:
weights_, mse_ = least_squares(y_train, x_train)

prediction = x_test @ weights_

transformed_prediction = transform_prediction(prediction)

print(compute_accuracy(y_train, transform_prediction(x_train @ weights_)) * 100)

print(compute_accuracy(y_test, transformed_prediction) * 100)

In [ ]:
## lambda = 5e-4 71.6ish%

weights_, mse_ = ridge_regression(y_train, x_train,1.7)

prediction = x_test @ weights_

transformed_prediction = transform_prediction(prediction)

print(compute_accuracy(y_train, transform_prediction(x_train @ weights_)) * 100)

print(compute_accuracy(y_test, transformed_prediction) * 100)

In [ ]:
## Uses gradient descent
def logistic_regression_newton(y, tx, initial_w, max_iters, gamma):
    # init parameters
    threshold = 1e-8
    losses = []

    w = initial_w

    # start the logistic regression with GD
    for iter in range(max_iters):
        # get loss and update w.
        loss = calculate_loss(y, tx, w) 
        hess = calculate_hessian(y, tx, w)
        w = np.linalg.solve(hess, hess @ w - gamma * calculate_gradient(y, tx, w))
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    
    
    losses.append(calculate_loss(y, tx, w))
    print("loss={l}".format(l=losses[-1]))
    
    return w, losses[-1]

## lambda = 5e-4 71.6ish%

N, D = x_train.shape

weights_, mse_ = logistic_regression_newton(y_train.reshape((N, 1)), x_train, np.zeros((D, 1)), 400, 5e-4)

prediction = x_test @ weights_

transformed_prediction = transform_prediction(prediction)

print(compute_accuracy(y_train.reshape((N, 1)), transform_prediction(x_train @ weights_)) * 100)

print(compute_accuracy(y_test.reshape((len(y_test), 1)), transformed_prediction) * 100)

## K-fold cross validation on lambda for logistic regression

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.
    
    Args:
        y:      shape=(N,)
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold

    >>> build_k_indices(np.array([1., 2., 3., 4.]), 2, 1)
    array([[3, 2],
           [0, 1]])
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation(y, x, k_indices, k, lambda_, gamma=0.1):
    """return the loss of ridge regression for a fold corresponding to k_indices
    
    Args:
        y:          shape=(N,)
        x:          shape=(N,D)
        k_indices:  2D array returned by build_k_indices()
        k:          scalar, the k-th fold (N.B.: not to confused with k_fold which is the fold nums)
        lambda_:    scalar, cf. ridge_regression()
        degree:     scalar, cf. build_poly()

    Returns:
        train and test root mean square errors rmse = sqrt(2 mse)

    >>> cross_validation(np.array([1.,2.,3.,4.]), np.array([6.,7.,8.,9.]), np.array([[3,2], [0,1]]), 1, 2, 3)
    (0.019866645527597114, 0.33555914361295175)
    """
    
    N, D = x.shape

    # ***************************************************
    # get k'th subgroup in test, others in train
    # ***************************************************
    k_te_indices = k_indices[k]
    te_mask = np.zeros(N, dtype = bool)
    te_mask[k_te_indices] = True
    
    y_te = y[te_mask]
    y_tr = y[~te_mask]
    
    x_te = x[te_mask]
    x_tr = x[~te_mask]
    
    # ***************************************************
    # Regularized logistic regression
    # ***************************************************
    
    np.random.seed(42)
    initial_weights = np.random.randn(D, 1)
    max_iters = 150
    weights, loss = reg_logistic_regression(y_tr, x_tr, lambda_, initial_weights, max_iters, gamma)
    
    # ***************************************************
    # calculate the loss for train and test data
    # ***************************************************
    
    loss_tr = np.sqrt(2 * compute_mse(y_tr, x_tr, weights))
    loss_te = np.sqrt(2 * compute_mse(y_te, x_te, weights))
    
    return loss_tr, loss_te

def cross_validation_demo(y, x, k_fold, lambdas):
    """cross validation over regularisation parameter lambda.
    
    Args:
        degree: integer, degree of the polynomial expansion
        k_fold: integer, the number of folds
        lambdas: shape = (p, ) where p is the number of values of lambda to test
    Returns:
        best_lambda : scalar, value of the best lambda
        best_rmse : scalar, the associated root mean squared error for the best lambda
    """
    
    seed = 12
    k_fold = k_fold
    lambdas = lambdas
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # ***************************************************
    # cross validation over lambdas
    # ***************************************************
    
    for lambda_ in lambdas:
        aux_tr = 0; aux_te = 0
        for k in np.arange(k_fold):
            loss_tr_tmp, loss_te_tmp = cross_validation(y, x, k_indices, k, lambda_)
            
            aux_tr += loss_tr_tmp
            aux_te += loss_te_tmp
            
        rmse_tr.append(aux_tr/k_fold)
        rmse_te.append(aux_te/k_fold)   

    ## Computing the best lambda & test rmse tuple
    best_idx = np.argmin(rmse_te)
    
    best_lambda = lambdas[best_idx]
    best_rmse   = rmse_te[best_idx]
        
    print("The choice of lambda which leads to the best test rmse is %.5f with a test rmse of %.3f" % (best_lambda, best_rmse))
    return best_lambda, best_rmse

In [ ]:
np.random.seed(42)
gamma = 0.1
lambda_ = 0.5

initial_weight = np.random.randn(D,1)

weights, loss = reg_logistic_regression(y_train.reshape((len(y_train), 1)), x_train, lambda_, initial_weight, 900, gamma)

print(f'After cross validation on the choice of lambda_, we obtain a loss of {calculate_loss(y_test.reshape((len(y_test), 1)), x_test, weights)} on the test set.')


prediction = x_test @ weights

transformed_prediction = transform_prediction(prediction)

print(compute_accuracy(y_train.reshape((len(y_train), 1)), transform_prediction(x_train @ weights)) * 100)

print(compute_accuracy(y_test.reshape((len(y_test), 1)), transformed_prediction) * 100)

In [ ]:
##Fn call
best_gamma, best_lambda, best_rmse = best_param_selection(y_train, tx_train, np.logspace(-4, 0, 30), 4, np.logspace(-4, 0, 30))

print(f'[Reg. Logistic Regression] In max_iter={max_iter}, with hyperparameters lambda={lambda_} and gamma={gamma} we obtain a loss={loss} and a test loss={5} TODO')